In [75]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from fractions import Fraction
directory = 'lorenz/'


# dyn_lorenz = {"sigma":16.0, "R":45.92, "b": 4.0}
dyn_lorenz = {"sigma":10, "R":28, "b": 8/3}
v1 = np.array([1.0, 0.0, 0.0])
v2 = np.array([0.0, 1.0, 0.0])
v3 = np.array([0.0, 0.0, 1.0])
# x_dot = []
# x_norm = []
# le_array = []

def lorenz (x0, dyn):
    sigma = dyn['sigma']
    R = dyn['R']
    b = dyn['b']
    x = x0[0]
    y = x0[1]
    z = x0[2]
    return np.array([sigma * (y - x),
                     x * (R - z) - y,
                     x * y - b * z])


fp1 = np.array([0,0,0])
fp2 = np.array([np.sqrt(dyn_lorenz['b']*(dyn_lorenz['R']-1)), np.sqrt(dyn_lorenz['b']*(dyn_lorenz['R']-1)), dyn_lorenz['R']-1])
fp3 = np.array([-np.sqrt(dyn_lorenz['b']*(dyn_lorenz['R']-1)), -np.sqrt(dyn_lorenz['b']*(dyn_lorenz['R']-1)), dyn_lorenz['R']-1])

def linearized_lorenz (x0, dyn, y_lorenz):
    sigma = dyn['sigma']
    R = dyn['R']
    b = dyn['b']
    x = y_lorenz[0]
    y = y_lorenz[1]
    z = y_lorenz[2]
    pre_dot = np.array([[-sigma, sigma, 0],
                        [R-z, -1, -x],
                        [y,x,-b]
                        ])
    af_dot = np.dot(pre_dot, x0)
    return af_dot

def RungeKutta (dyn, f, dt, x0):
    k1 = f(x0, dyn) #[x,y,z]*0.1 example
    k2 = f(x0+0.5*k1*dt,dyn)
    k3 = f(x0 + 0.5*k2*dt, dyn)
    k4 = f(x0 + k3*dt, dyn)

    x = x0 + ((k1 + 2*k2 + 2*k3 + k4)/6) *dt
    return x

def RungeKutta_linearized (dyn, f, dt, x0, y):
    k1 = f(x0, dyn, y) #[x,y,z]*0.1 example
    k2 = f(x0+0.5*k1*dt,dyn, y)
    k3 = f(x0 + 0.5*k2*dt, dyn, y)
    k4 = f(x0 + k3*dt, dyn, y)

    x = x0 + ((k1 + 2*k2 + 2*k3 + k4)/6) * dt

    return x


def f_t_le (dyn, f, linearized_f, dt, x0, T):
    x = np.empty(shape=(len(x0),T))
    v1_prime = np.empty(shape=(len(x0),T))
    v2_prime = np.empty(shape=(len(x0),T))
    v3_prime = np.empty(shape=(len(x0),T))
    x[:, 0] = x0
    v1_prime[:, 0] = v1
    v2_prime[:, 0] = v2
    v3_prime[:, 0] = v3
    le = np.array([0,0,0])
    for i in range(1,T):
        x[:, i] = RungeKutta(dyn, f, dt, x[:, i-1])

        v1_prime[:, i] = RungeKutta_linearized(dyn, linearized_f, dt, v1_prime[:, i-1], x[:, i-1])
        v2_prime[:, i] = RungeKutta_linearized(dyn, linearized_f, dt, v2_prime[:, i-1], x[:, i-1])
        v3_prime[:, i] = RungeKutta_linearized(dyn, linearized_f, dt, v3_prime[:, i-1], x[:, i-1])


        norm1 = np.linalg.norm(v1_prime[:, i])
        v1_prime[:, i] = v1_prime[:, i]/norm1

        GSC1 = np.dot(v2_prime[:, i], v1_prime[:, i])
        v2_prime[:, i] = v2_prime[:, i] - GSC1*v1_prime[:, i]

        norm2 = np.linalg.norm(v2_prime[:, i])
        v2_prime[:, i] = v2_prime[:, i]/norm2

        GSC2 = np.dot(v3_prime[:, i], v1_prime[:, i])
        GSC3 = np.dot(v3_prime[:, i], v2_prime[:, i])

        v3_prime[:, i] = v3_prime[:, i] - GSC2*v1_prime[:, i] - GSC3*v2_prime[:, i]
        norm3 = np.linalg.norm(v3_prime[:, i])
        v3_prime[:, i] = v3_prime[:, i]/norm3

        le = le + np.log(np.array([norm1,norm2,norm3]))




    return x, le/(T*dt)

# def f_t (dyn, f, dt, x0, T):
#     x = np.empty(shape=(len(x0),T))
#     x[:, 0] = x0
#     for i in range(1,T):
#         x[:, i] = RungeKutta(dyn, f, dt, x[:, i-1])
#     return x

In [76]:
x,le = f_t_le(dyn_lorenz, lorenz, linearized_lorenz, 0.01, np.array([1.0, 0.0, 1.0]), 200000)

In [77]:
le

array([ 9.00771714e-01, -1.30963945e-03, -1.45659558e+01])

In [60]:
T = 500
dt = 0.001
dyn_lorenz = {"sigma":10, "R":28, "b": 8/3}
dyn = dyn_lorenz
f = lorenz
K = 20000
delta0 = 1e-4
x0 = np.array([1.0, 0.0, 1.0])
I1 = np.array([1.0, 0.0, 0.0])
I2 = np.array([0.0, 1.0, 0.0])
I3 = np.array([0.0, 0.0, 1.0])
# x = f_t (dyn, f, dt, x0, T)
# xc1 = f_t (dyn, f, dt, x0c1, T)
# xc2 = f_t (dyn, f, dt, x0c2, T)
# xc3 = f_t (dyn, f, dt, x0c2, T)

In [66]:
def f_t (dyn, f, dt, x0, T, K, delta0):
    x = np.empty(shape=(len(x0),T))
    xc1 = np.empty(shape=(len(x0),T))
    xc2 = np.empty(shape=(len(x0),T))
    xc3 = np.empty(shape=(len(x0),T))
    x0c1 = x0 + I1*delta0
    x0c2 = x0 + I2*delta0
    x0c3 = x0 + I3*delta0
    le = np.array([0,0,0])
    for j in range(0,K):

        xc1[:, 0] = x0c1
        xc2[:, 0] = x0c2
        xc3[:, 0] = x0c3
        x[:, 0] = x0
        for i in range(1,T):
            x[:, i] = RungeKutta(dyn, f, dt, x[:, i-1])
            xc1[:, i] = RungeKutta(dyn, f, dt, xc1[:, i-1])
            xc2[:, i] = RungeKutta(dyn, f, dt, xc2[:, i-1])
            xc3[:, i] = RungeKutta(dyn, f, dt, xc3[:, i-1])
        
        delta1x = x[:, -1] - xc1[:, -1]
        delta2x = x[:, -1] - xc2[:, -1]
        delta3x = x[:, -1] - xc3[:, -1]
#         print (delta1x)
        
        v1 = delta1x
        norm1 = np.linalg.norm(delta1x)
        u1 = delta1x/norm1

        GSC1 = np.dot(delta2x, u1)
        v2 = delta2x - GSC1*u1

        norm2 = np.linalg.norm(v2)
        u2 = v2/norm2

        GSC2 = np.dot(delta3x, u1)
        GSC3 = np.dot(delta3x, u2)

        v3 = delta3x - GSC2*u1 - GSC3*u2
        norm3 = np.linalg.norm(v3)
        u3 = v3/norm3
        
        x0c1 = x[:, -1]  + u1*delta0
        x0c2 = x[:, -1]  + u2*delta0
        x0c3 = x[:, -1]  + u3*delta0
        x0 = x[:, -1] 
        
        le = le + np.log(np.array([norm1,norm2,norm3])/delta0)
#         print(norm1)

    
    return le/(K*T*dt)

In [67]:
le_sample = f_t (dyn, f, dt, x0, T, K, delta0)

In [68]:
le_sample

array([ 9.05032248e-01, -4.52416135e-04, -1.45232011e+01])